In [20]:
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from eval_metrics import compute_metrics
from aim.hugging_face import AimCallback
from text_format_utils import generate_formatted_string
import json


def process_str(str):
    # string = """{\"synonyms\":[{\"name\":\"p-Phenylazo carbanilic acid, n-hexyl ester\"}],\"related\":[{\"SMILES\":\"CCCCOC(=O)NC1=CC=C(C=C1)N\",\"similarity\":0.74}],\"experimental\":[],\"CID\":523129,\"SMILES\":\"CCCCCCOC(=O)NC1=CC=C(C=C1)N=NC2=CC=CC=C2\",\"SAS\":2.09,\"WEIGHT\":325.18,\"TPSA\":63.05,\"CLOGP\":6.23,\"QED\":0.46,\"NUMHDONORS\":1,\"NUMHACCEPTORS\":4,\"NUMHETEROATOMS\":5,\"NUMROTATABLEBONDS\":8,\"NOCOUNT\":5,\"NHOHCOUNT\":1,\"RINGCOUNT\":2,\"HEAVYATOMCOUNT\":24,\"FRACTIONCSP3\":0.32,\"NUMAROMATICRINGS\":2,\"NUMSATURATEDRINGS\":0,\"NUMAROMATICHETEROCYCLES\":0,\"NUMAROMATICCARBOCYCLES\":2,\"NUMSATURATEDHETEROCYCLES\":0,\"NUMSATURATEDCARBOCYCLES\":0,\"NUMALIPHATICRINGS\":0,\"NUMALIPHATICHETEROCYCLES\":0,\"NUMALIPHATICCARBOCYCLES\":0,\"IUPAC\":\"hexyl N-(4-phenyldiazenylphenyl)carbamate\"}"""

    print(str)
    # str["text"] = str["text"].replace('\\\', '\\'')
    # print(string)
    com = json.loads(json.loads((str["text"])))
    str['txt'] = generate_formatted_string(com)
    print(str['txt'])
    print(com)
    print('done')
    return str


def tokenize_function(examples):
    return tokenizer(examples["text"])


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


def tokenize_function(examples):
    return tokenizer(examples["text"])


if __name__ == "__main__":
    model_checkpoint = "facebook/galactica-125m"
    block_size = 128

    model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
                               
    dataset = load_dataset(
        "text",
        data_files={"train": "./train.jsonl", "validation": "./evaluation.jsonl"},
        streaming=True,
    )
    print('data laoded')
    dataset = dataset.map(process_str)
    # tokenized_datasets = dataset.map(
    #     tokenize_function, batched=True, remove_columns=["text"]
    # )
    # lm_datasets = tokenized_datasets.map(group_texts, batched=True, batch_size=1000)

    # aim_callback = AimCallback(repo=".", experiment="your_experiment_name")

    training_args = TrainingArguments(
        output_dir=f"{model_checkpoint.split('/')[-1]}-finetuned-pubchem",
        evaluation_strategy="steps",
        learning_rate=6e-6,
        lr_scheduler_type='linear',
        weight_decay=0.1,
        adam_beta1=0.9,
        adam_beta2=0.95,
        warmup_steps=500,
        max_grad_norm=1.0,
        eval_steps=1,
        max_steps=10,
        num_train_epochs=5
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        # compute_metrics=compute_metrics,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        # callbacks=[aim_callback],
    )

    trainer.train()


data laoded
{'text': '"{\\"synonyms\\":[{\\"name\\":\\"p-Phenylazo carbanilic acid, n-hexyl ester\\"}],\\"related\\":[{\\"SMILES\\":\\"CCCCOC(=O)NC1=CC=C(C=C1)N\\",\\"similarity\\":0.74}],\\"experimental\\":[],\\"CID\\":523129,\\"SMILES\\":\\"CCCCCCOC(=O)NC1=CC=C(C=C1)N=NC2=CC=CC=C2\\",\\"SAS\\":2.09,\\"WEIGHT\\":325.18,\\"TPSA\\":63.05,\\"CLOGP\\":6.23,\\"QED\\":0.46,\\"NUMHDONORS\\":1,\\"NUMHACCEPTORS\\":4,\\"NUMHETEROATOMS\\":5,\\"NUMROTATABLEBONDS\\":8,\\"NOCOUNT\\":5,\\"NHOHCOUNT\\":1,\\"RINGCOUNT\\":2,\\"HEAVYATOMCOUNT\\":24,\\"FRACTIONCSP3\\":0.32,\\"NUMAROMATICRINGS\\":2,\\"NUMSATURATEDRINGS\\":0,\\"NUMAROMATICHETEROCYCLES\\":0,\\"NUMAROMATICCARBOCYCLES\\":2,\\"NUMSATURATEDHETEROCYCLES\\":0,\\"NUMSATURATEDCARBOCYCLES\\":0,\\"NUMALIPHATICRINGS\\":0,\\"NUMALIPHATICHETEROCYCLES\\":0,\\"NUMALIPHATICCARBOCYCLES\\":0,\\"IUPAC\\":\\"hexyl N-(4-phenyldiazenylphenyl)carbamate\\"}"'}
<class 'dict'>
{'synonyms': [{'name': 'p-Phenylazo carbanilic acid, n-hexyl ester'}], 'related': [{'SMILE

/home/menuab/miniconda3/envs/pubchem/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,head_mask,past_key_values,inputs_embeds,labels,use_cache,output_attentions,output_hidden_states,return_dict,label_ids,label,labels.

In [5]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets['train'],
    eval_dataset=lm_datasets["validation"],
)

trainer.train()

/home/menuab/miniconda3/envs/pubchem/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/menuab/miniconda3/envs/pubchem/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,1.623429


TrainOutput(global_step=2, training_loss=2.262911796569824, metrics={'train_runtime': 92.3246, 'train_samples_per_second': 0.347, 'train_steps_per_second': 0.022, 'total_flos': 2090336256000.0, 'train_loss': 2.262911796569824, 'epoch': 1.0})

In [69]:
import pyarrow as pa
from pyarrow import json
    
raw_schema = pa.schema([
    pa.field('SomeDecimal', pa.string())
])

In [82]:
schema = {'synonyms': [{'name': 'p-Phenylazo carbanilic acid, n-hexyl ester'}],
 'related': [{'SMILES': 'CCCCOC(=O)NC1=CC=C(C=C1)N', 'similarity': 0.74}],
 'experimental': [],
 'CID': 523129,
 'SMILES': 'CCCCCCOC(=O)NC1=CC=C(C=C1)N=NC2=CC=CC=C2',
 'SAS': 2.09,
 'WEIGHT': 325.18,
 'TPSA': 63.05,
 'CLOGP': 6.23,
 'QED': 0.46,
 'NUMHDONORS': 1,
 'NUMHACCEPTORS': 4,
 'NUMHETEROATOMS': 5,
 'NUMROTATABLEBONDS': 8,
 'NOCOUNT': 5,
 'NHOHCOUNT': 1,
 'RINGCOUNT': 2,
 'HEAVYATOMCOUNT': 24,
 'FRACTIONCSP3': 0.32,
 'NUMAROMATICRINGS': 2,
 'NUMSATURATEDRINGS': 0,
 'NUMAROMATICHETEROCYCLES': 0,
 'NUMAROMATICCARBOCYCLES': 2,
 'NUMSATURATEDHETEROCYCLES': 0,
 'NUMSATURATEDCARBOCYCLES': 0,
 'NUMALIPHATICRINGS': 0,
 'NUMALIPHATICHETEROCYCLES': 0,
 'NUMALIPHATICCARBOCYCLES': 0,
 'IUPAC': 'hexyl N-(4-phenyldiazenylphenyl)carbamate'}

In [89]:
s = pa.array(schema)

TypeError: 'str' object cannot be interpreted as an integer

In [95]:
l=[]
for k,v in schema.items():
    f = k
    if isinstance(v,list):
        s = pa.list_(pa.dictionary(pa.int16(),pa.string()))
    elif isinstance(v, str):
        s = pa.string()
    else:
        s = pa.float16()

    l.append(pa.field(f,s))

In [84]:
raw_schema = pa.schema(l)

In [85]:
raw_schema

synonyms: list<item: dictionary<values=string, indices=int16, ordered=0>>
  child 0, item: dictionary<values=string, indices=int16, ordered=0>
related: list<item: dictionary<values=string, indices=int16, ordered=0>>
  child 0, item: dictionary<values=string, indices=int16, ordered=0>
experimental: list<item: dictionary<values=string, indices=int16, ordered=0>>
  child 0, item: dictionary<values=string, indices=int16, ordered=0>
CID: halffloat
SMILES: string
SAS: halffloat
WEIGHT: halffloat
TPSA: halffloat
CLOGP: halffloat
QED: halffloat
NUMHDONORS: halffloat
NUMHACCEPTORS: halffloat
NUMHETEROATOMS: halffloat
NUMROTATABLEBONDS: halffloat
NOCOUNT: halffloat
NHOHCOUNT: halffloat
RINGCOUNT: halffloat
HEAVYATOMCOUNT: halffloat
FRACTIONCSP3: halffloat
NUMAROMATICRINGS: halffloat
NUMSATURATEDRINGS: halffloat
NUMAROMATICHETEROCYCLES: halffloat
NUMAROMATICCARBOCYCLES: halffloat
NUMSATURATEDHETEROCYCLES: halffloat
NUMSATURATEDCARBOCYCLES: halffloat
NUMALIPHATICRINGS: halffloat
NUMALIPHATICHETERO

In [81]:
json.read_json('../dataloader/523129_start.jsonl', parse_options=json.ParseOptions(explicit_schema=raw_schema))

ArrowNotImplementedError: JSON conversion to halffloat is not supported